In [32]:
# install dependencies
!pip3 install snscrape
!pip install vaderSentiment
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
# import libraries

import numpy as np
import pandas as pd

import snscrape.modules.twitter as sntwitter
import snscrape.modules.twitter

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

import matplotlib.pyplot as mlpt
import csv
import datetime as dt
import pyspark as spark

import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re


In [34]:
pre_tweets_list = []

# use TwitterSearchScraper to scrape data and append tweets to list
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('#turkey since:2023-02-05_00:00:00_UTC- until:2023-02-05_23:59:59_UTC lang:en').get_items()):
    if i>=1000:
      break
    pre_tweets_list.append([tweet.date, tweet.rawContent])

pre_tweets_df = pd.DataFrame(pre_tweets_list, columns=['date', 'content'])
pre_tweets_df_orig = pre_tweets_df
pre_tweets_df

,date,content
0,2023-02-05 23:58:11+00:00,This account for meeting between #Arabic Speak...
1,2023-02-05 23:53:07+00:00,"Pervin Buldan, #HDP Co-Chair, says party will ..."
2,2023-02-05 23:49:09+00:00,Maximize efficiency &amp; minimize costs in dr...
3,2023-02-05 23:44:01+00:00,"Today is the feast day of Saint Polyeuctus, 10..."
4,2023-02-05 23:39:03+00:00,One of a kind hair transplant training course ...
...,...,...
496,2023-02-05 00:15:59+00:00,#Earthquake (#deprem) possibly felt 16 sec ago...
497,2023-02-05 00:05:24+00:00,"So, it seems #Turkey army personnel is activel..."
498,2023-02-05 00:01:11+00:00,Shrouded In Mystery &amp; Obscurity – Discover...
499,2023-02-05 00:00:01+00:00,📌The pro-government media outlets in #Turkey c...


In [35]:
post_tweets_list = []

# use TwitterSearchScraper to scrape data and append tweets to list
i = 0
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('#turkey since:2023-02-07_00:00:00_UTC- until:2023-02-07_23:59:59_UTC lang:en').get_items()):
    if i>=1000:
      break
    post_tweets_list.append([tweet.date, tweet.rawContent])

post_tweets_df = pd.DataFrame(post_tweets_list, columns=['date', 'content'])
post_tweets_df_orig = post_tweets_df
post_tweets_df

,date,content
0,2023-02-07 23:59:52+00:00,🇸🇾🇹🇷\nI pray for the lives affected and the so...
1,2023-02-07 23:59:48+00:00,Move the victim away from the place of danger ...
2,2023-02-07 23:59:47+00:00,An estimate in The Economist predicts 180k dea...
3,2023-02-07 23:59:33+00:00,#Turkey\nवसुधैव कुटुंबकम्।\nThe World is One F...
4,2023-02-07 23:59:31+00:00,#Earthquake (#deprem) M2.8 strikes 23 km E of ...
...,...,...
995,2023-02-07 21:49:41+00:00,"#Haarp_Project It works! Next , China. #Turkey"
996,2023-02-07 21:49:40+00:00,Let's pray for #Syria &amp; #Turkey https://t....
997,2023-02-07 21:49:35+00:00,#MaxSports |\n\nTurkish goalie Eyup Turkaslan ...
998,2023-02-07 21:49:19+00:00,Race to find survivors as quake aid pours into...


In [36]:
pre_tweets_df = pre_tweets_df_orig

data=pre_tweets_df
cdata=pd.DataFrame(columns=['date','content'])
index=0
for index,row in data.iterrows():
    stre=row["content"]
    #the following cleaning steps are not the standard. You may generate your own ways of cleaning.
    my_new_string = re.sub('#\w+', '', stre) #remove the hashtags
    my_new_string = re.sub('http\S+', '', my_new_string) #remove the links
    my_new_string = re.sub('\n', '', my_new_string) #remove \n
    cdata.sort_index()
    cdata.at[index,'date']=row["date"]
    cdata.at[index,'content']=my_new_string
    index=index+1
pre_tweets_df=cdata
# pre_tweets_df['date'] = pre_tweets_df['date']

# pre_tweets_df['date'] = pre_tweets_df['date'].astype('datetime64[ns]')
# # pre_tweets_df["date"] = pre_tweets_df["date"].dt.date

pre_tweets_df.head()

,date,content
0,2023-02-05 23:58:11+00:00,This account for meeting between Speakers in
1,2023-02-05 23:53:07+00:00,"Pervin Buldan, Co-Chair, says party will run ..."
2,2023-02-05 23:49:09+00:00,Maximize efficiency &amp; minimize costs in dr...
3,2023-02-05 23:44:01+00:00,"Today is the feast day of Saint Polyeuctus, 10..."
4,2023-02-05 23:39:03+00:00,One of a kind hair transplant training course ...


In [37]:

post_tweets_df = post_tweets_df_orig

data=post_tweets_df
cdata=pd.DataFrame(columns=['date','content'])
index=0
for index,row in data.iterrows():
    stre=row["content"]
    #the following cleaning steps are not the standard. You may generate your own ways of cleaning.
    my_new_string = re.sub('#\w+', '', stre) #remove the hashtags
    my_new_string = re.sub('http\S+', '', my_new_string) #remove the links
    my_new_string = re.sub('\n', '', my_new_string) #remove \n
    cdata.sort_index()
    cdata.at[index,'date']=row["date"]
    cdata.at[index,'content']=my_new_string
    index=index+1
post_tweets_df=cdata
# post_tweets_df['date'] = post_tweets_df['date']

# post_tweets_df['date'] = post_tweets_df['date'].astype('datetime64[ns]')
# # post_tweets_df["date"] = post_tweets_df["date"].dt.date

post_tweets_df.head()

,date,content
0,2023-02-07 23:59:52+00:00,🇸🇾🇹🇷I pray for the lives affected and the soul...
1,2023-02-07 23:59:48+00:00,Move the victim away from the place of danger ...
2,2023-02-07 23:59:47+00:00,An estimate in The Economist predicts 180k dea...
3,2023-02-07 23:59:33+00:00,वसुधैव कुटुंबकम्।The World is One Family.India...
4,2023-02-07 23:59:31+00:00,() M2.8 strikes 23 km E of () 9 min ago. Mor...


In [38]:
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

ValueError: ignored

In [28]:
# import pytz
# utc=pytz.UTC

# pre_tweets_df['orig_date'] = pre_tweets_df['date']
# pre_tweets_df['date'] = pre_tweets_df['date'].apply(lambda x : x.to_pydatetime())

# post_tweets_df['orig_date'] = post_tweets_df['date']
# post_tweets_df['date'] = post_tweets_df['date'].apply(lambda x : x.to_pydatetime())

AttributeError: ignored

In [25]:
def generateSentiment(tweets_df):

  FullDataTw=sql.createDataFrame(tweets_df)
  FullDataTw = FullDataTw.dropna()
  FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")

  CleanDF = FullDataTw
  from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
  analyser = SentimentIntensityAnalyzer()
  def senti_score_udf(sentence):
      snt = analyser.polarity_scores(sentence)
      return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])

  func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
  CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['content'])[0])  #withColumn function would generate the result with a new column name
  CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['content'])[1])
  CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['content'])[2])
  CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['content'])[3])
  CleanDF.show(10)
  # wc_tweets_df = CleanDF.toPandas().to_csv('sen.csv',index=False)
  tweets_df = CleanDF.toPandas()
  return tweets_df

In [41]:
pre_tweets_df = generateSentiment(pre_tweets_df)
pre_tweets_df.head()

+----+--------------------+-----+-----+-----+-------+
|date|             content|p_neg|p_neu|p_pos| p_comp|
+----+--------------------+-----+-----+-----+-------+
|  {}|This account for ...|  0.0|  1.0|  0.0|    0.0|
|  {}|Pervin Buldan,  C...|0.133|0.644|0.222| 0.4614|
|  {}|Maximize efficien...|  0.0|0.773|0.227| 0.8176|
|  {}|Today is the feas...|  0.0|  1.0|  0.0|    0.0|
|  {}|One of a kind hai...|  0.0|0.764|0.236| 0.5267|
|  {}|Simon Tisdall is ...|0.117|0.785|0.098|-0.2091|
|  {}|📌The 36-member  ...|0.244|0.595|0.161|-0.3944|
|  {}|📌The 36-member  ...|0.244|0.595|0.161|-0.3944|
|  {}|'s President  say...|0.167|0.833|  0.0|-0.1027|
|  {}|What will  do whe...|0.087|0.856|0.056|-0.3578|
+----+--------------------+-----+-----+-----+-------+
only showing top 10 rows



,date,content,p_neg,p_neu,p_pos,p_comp
0,(),This account for meeting between Speakers in,0.000,1.000,0.000,0.0000
1,(),"Pervin Buldan, Co-Chair, says party will run ...",0.133,0.644,0.222,0.4614
2,(),Maximize efficiency &amp; minimize costs in dr...,0.000,0.773,0.227,0.8176
3,(),"Today is the feast day of Saint Polyeuctus, 10...",0.000,1.000,0.000,0.0000
4,(),One of a kind hair transplant training course ...,0.000,0.764,0.236,0.5267


In [42]:
post_tweets_df = generateSentiment(post_tweets_df)
post_tweets_df.head()

+----+--------------------+-----+-----+-----+-------+
|date|             content|p_neg|p_neu|p_pos| p_comp|
+----+--------------------+-----+-----+-----+-------+
|  {}|🇸🇾🇹🇷I pray fo...|0.203|0.677| 0.12|-0.1531|
|  {}|Move the victim a...|0.194|0.582|0.225| 0.4404|
|  {}|An estimate in Th...|0.273|0.727|  0.0|-0.9432|
|  {}|वसुधैव कुटुंबकम्।...|  0.0|  1.0|  0.0|    0.0|
|  {}| () M2.8 strikes ...|0.172|0.828|  0.0|-0.3612|
|  {}|Pls help            |  0.0|  0.0|  1.0| 0.4588|
|  {}|44 hours after th...|  0.0|0.865|0.135| 0.4215|
|  {}|Death toll from -...|0.438|0.562|  0.0|-0.5994|
|  {}|What a terrible d...|0.399|0.552|0.049|-0.9477|
|  {}|The Norwegian Ref...|  0.0|0.777|0.223| 0.3182|
+----+--------------------+-----+-----+-----+-------+
only showing top 10 rows



,date,content,p_neg,p_neu,p_pos,p_comp
0,(),🇸🇾🇹🇷I pray for the lives affected and the soul...,0.203,0.677,0.120,-0.1531
1,(),Move the victim away from the place of danger ...,0.194,0.582,0.225,0.4404
2,(),An estimate in The Economist predicts 180k dea...,0.273,0.727,0.000,-0.9432
3,(),वसुधैव कुटुंबकम्।The World is One Family.India...,0.000,1.000,0.000,0.0000
4,(),() M2.8 strikes 23 km E of () 9 min ago. Mor...,0.172,0.828,0.000,-0.3612


In [43]:
from google.colab import files
pre_tweets_df.to_csv('pre_tweets_df.csv', encoding = 'utf-8-sig') 
files.download('pre_tweets_df.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
post_tweets_df.to_csv('post_tweets_df.csv', encoding = 'utf-8-sig') 
files.download('post_tweets_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>